# MLflow and TensorBoard Examples

This notebook demonstrates basic usage of MLflow and TensorBoard for experiment tracking.

## MLflow Example

This example shows how to log parameters, metrics and artifacts using MLflow.

In [ ]:
# Simple MLflow Example
import mlflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Set the experiment name
mlflow.set_experiment("Simple-Linear-Regression")

# Create sample data
np.random.seed(42)
X = pd.DataFrame(np.random.rand(100, 1), columns=['feature'])
y = 2 * X.squeeze() + 1 + 0.1 * np.random.randn(100)

# Start an MLflow run
with mlflow.start_run() as run:
    # Log parameters
    mlflow.log_param("data_size", len(X))
    mlflow.log_param("random_seed", 42)
    
    # Train model
    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    mse = np.mean((predictions - y) ** 2)
    
    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("slope", model.coef_[0])
    mlflow.log_metric("intercept", model.intercept_)
    
    # Create and log a plot
    plt.figure(figsize=(8, 5))
    plt.scatter(X, y, alpha=0.5)
    plt.plot(X, predictions, color='red', linewidth=2)
    plt.title(f"Linear Regression (MSE: {mse:.4f})")
    plt.xlabel("X")
    plt.ylabel("y")
    plt.savefig("regression_plot.png")
    mlflow.log_artifact("regression_plot.png")
    
    # Log model
    input_example = X[:1] 
    mlflow.sklearn.log_model(model, name="model", input_example=input_example)
    
    # Register model to view it under model uri
    model_uri = f"runs:/{run.info.run_id}/model"
    mlflow.register_model(model_uri, "test")
    
    print(f"Run ID: {run.info.run_id}")
    print(f"MSE: {mse:.4f}")
    print(f"Slope: {model.coef_[0]:.4f}, Intercept: {model.intercept_:.4f}")
    print(f"Open MLflow extension to view results!")

## TensorBoard Example

This example shows how to log metrics and visualizations using the custom TensorBoard logger.

In [ ]:
# Simple TensorBoard Example using custom logger
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Import the custom TensorBoard logger
from tensorboard_logger import TensorBoardLogger

# Create TensorBoard logger with a specific experiment name
tb_logger = TensorBoardLogger(experiment_name="simple_neural_network")
tb_logger.summary()  # Display logging information

# Create sample data
np.random.seed(42)
X = np.random.normal(0, 1, (1000, 10))
y = (X[:, 0] > 0).astype(int)  # Binary classification based on first feature

# Split data
X_train, X_test = X[:800], X[800:]
y_train, y_test = y[:800], y[800:]

# Create a simple model
model = Sequential([
    Dense(16, activation='relu', input_shape=(10,)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Get the TensorBoard callback from our custom logger
tensorboard_callback = tb_logger.get_callback()

# Train the model with TensorBoard callback
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {accuracy:.4f}")

# The TensorBoard logs can be accessed via the JupyterHub TensorBoard extension

## Using Both Tools Together

You can use both MLflow and TensorBoard in your workflow:
- MLflow for tracking experiments, parameters, and models
- TensorBoard for detailed training visualizations

Both tools are available as extensions in your JupyterHub environment.